In [49]:
from selenium import webdriver


driver = webdriver.Chrome()
driver.get("https://www.naver.com")



In [42]:
# driver.back()


In [43]:
# driver.forward()


In [36]:
driver.current_url

'https://www.google.com/'

In [61]:
driver.title






'NAVER'

In [70]:
#query
from selenium.webdriver.common.by import By

# CSS 셀렉터로 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, "#query")




In [71]:
# 검색창 클릭
search.click()


# 검색어 입력
search.send_keys("연세대학교")

from selenium.webdriver.common.keys import Keys  # Keys 클래스 import 추가
# 검색 실행을 위해 엔터키 입력 
search.send_keys(Keys.ENTER)

# 검색 결과 상위 5개 항목 찾기
results = driver.find_elements(By.CSS_SELECTOR, "#main .item")[:5]

# 검색 결과 출력
for result in results:
    print(result.text)



In [78]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys  # Keys 클래스 import 추가


driver = webdriver.Chrome()
driver.get("https://www.naver.com")

# driver.refresh()


# CSS 셀렉터로 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, "#query")

# 검색창 클릭
search.click()


# 검색어 입력
search.send_keys("연세대학교")

from selenium.webdriver.common.keys import Keys  # Keys 클래스 import 추가
# 검색 실행을 위해 엔터키 입력 
search.send_keys(Keys.ENTER)



In [75]:



# 해외축구(EPL) URL: 
# https://m.sports.naver.com/wfootball/schedule/index?category=epl&date=2024-11-23
# EPL cheer 예제
# https://m.sports.naver.com/game/2024111110050850171/cheer

#NBA URL
#https://m.sports.naver.com/basketball/schedule/index?category=nba&date=2024-11-22

#NBA URL cheer 예제
# https://m.sports.naver.com/game/2024111202/cheer

In [77]:
import requests

def check_robots_txt(url):
    try:
        response = requests.get(f"{url}/robots.txt")
        print(response.text)
    except Exception as e:
        print(f"에러 발생: {e}")

# 네이버 스포츠 robots.txt 확인
check_robots_txt("https://m.sports.naver.com")

User-agent: *
Disallow: /
Allow: /$
Allow: /index



In [ ]:

def crawl_schedule(category='epl'):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)
    
    try:
        today = datetime.now().strftime("%Y-%m-%d")
        url = f"https://m.sports.naver.com/wfootball/schedule/index?category={category}&date={today}"
        driver.get(url)
        time.sleep(2)
        
        matches = []
        game_elements = driver.find_elements(By.CSS_SELECTOR, ".game_item")
        
        for game in game_elements:
            try:
                game_id = game.get_attribute("data-game-id")
                home_team = game.find_element(By.CSS_SELECTOR, ".home_team").text
                away_team = game.find_element(By.CSS_SELECTOR, ".away_team").text
                match_time = game.find_element(By.CSS_SELECTOR, ".time").text
                
                match_info = {
                    "game_id": game_id,
                    "league": "EPL" if category == "epl" else "NBA",
                    "home_team": home_team,
                    "away_team": away_team,
                    "match_time": match_time,
                    "cheer_url": f"https://m.sports.naver.com/game/{game_id}/cheer"
                }
                matches.append(match_info)
                
            except Exception as e:
                print(f"매치 파싱 오류: {e}")
                
        return matches
    finally:
        driver.quit()


#### 해외축구(EPL) URL: 
##### https://m.sports.naver.com/wfootball/schedule/index?category=epl&date=2024-11-23

#### NBA URL:
##### https://m.sports.naver.com/basketball/schedule/index?category=nba&date=2024-11-22

#### EPL cheer 예제:
##### https://m.sports.naver.com/game/2024111110050850171/cheer

#### NBA cheer 예제:
##### https://m.sports.naver.com/game/2024111202/cheer

In [80]:
## 여기서부터 진짜 크롤링

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys  # Keys 클래스 import 추가
from selenium.webdriver.chrome.options import Options
import time
import json
from datetime import datetime



In [129]:

driver = webdriver.Chrome()

today = datetime.now()
category = "epl"

current_month = today.month
current_year = today.year

for i in range(6):  # 현재 달부터 5개월 후까지
    target_month = (current_month + i) % 12 or 12  # 12월 다음은 1월
    target_year = current_year + (current_month + i - 1) // 12  # 연도 변경 처리
    
    date = datetime(target_year, target_month, 1).strftime("%Y-%m-%d")
    url = f"https://m.sports.naver.com/wfootball/schedule/index?category={category}&date={date}"
    driver.get(url)
    time.sleep(2)
    elements = driver.find_elements(By.CSS_SELECTOR, ".ScheduleLeagueType_title__2Kalm")
    match_elements = driver.find_elements(By.CSS_SELECTOR, ".ScheduleLeagueType_match_list__1-n6x")
    for date_element, match_element in zip(elements, match_elements):
        print(f"\n[{date_element.text}] 경기일정:")  # 날짜 출력
        # li 요소들 찾기
        li_elements = match_element.find_elements(By.CSS_SELECTOR, "li")
        for idx, li in enumerate(li_elements):
            match_time = li.find_element(By.CSS_SELECTOR, ".MatchBox_time__nIEfd")
            match_status = li.find_element(By.CSS_SELECTOR, ".MatchBox_status__2pbzi")
            match_area = li.find_element(By.CSS_SELECTOR, ".MatchBox_match_area__39dEr")
            match_areas = match_area.find_elements(By.CSS_SELECTOR, ".MatchBoxTeamArea_team__3aB4O")
            link = match_area.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
            actual_time = match_time.get_attribute("textContent").strip().split("\n")[-1][-5:]

            print(f"경기날짜: {date_element.text}")
            print(f"경기 시간: {actual_time}")
            print(f"경기 상태: {match_status.text}")
            print(f"매치 정보: 홈 {match_areas[0].text} vs 원정 {match_areas[1].text}")
            print(f"응원 링크: {link}/cheer")
            print("-" * 50)
    time.sleep(1)

#content > div > div:nth-child(5) > div > div:nth-child(1) > ul > li:nth-child(1) > div > div.MatchBox_match_area__39dEr






[11월 2일 (토)] 경기일정:
경기날짜: 11월 2일 (토)
경기 시간: 21:30
경기 상태: 종료
매치 정보: 홈 뉴캐슬 vs 원정 아스널
응원 링크: https://m.sports.naver.com/game/2024110210050850157/cheer
--------------------------------------------------

[11월 3일 (일)] 경기일정:
경기날짜: 11월 3일 (일)
경기 시간: 00:00
경기 상태: 종료
매치 정보: 홈 본머스 vs 원정 맨시티
응원 링크: https://m.sports.naver.com/game/2024110310050850147/cheer
--------------------------------------------------
경기날짜: 11월 3일 (일)
경기 시간: 00:00
경기 상태: 종료
매치 정보: 홈 입스위치 vs 원정 레스터 시티
응원 링크: https://m.sports.naver.com/game/2024110310050850151/cheer
--------------------------------------------------
경기날짜: 11월 3일 (일)
경기 시간: 00:00
경기 상태: 종료
매치 정보: 홈 리버풀 vs 원정 브라이턴
응원 링크: https://m.sports.naver.com/game/2024110310050850153/cheer
--------------------------------------------------
경기날짜: 11월 3일 (일)
경기 시간: 00:00
경기 상태: 종료
매치 정보: 홈 노팅엄 vs 원정 웨스트햄
응원 링크: https://m.sports.naver.com/game/2024110310050850159/cheer
--------------------------------------------------
경기날짜: 11월 3일 (일)
경기 시간: 00:00
경기 상태: 종료
매치 정보: 홈 사우샘프턴 vs 원

In [132]:

driver = webdriver.Chrome()

today = datetime.now()
category = "nba"

current_month = today.month
current_year = today.year

for i in range(6):  # 현재 달부터 5개월 후까지
    target_month = (current_month + i) % 12 or 12  # 12월 다음은 1월
    target_year = current_year + (current_month + i - 1) // 12  # 연도 변경 처리
    
    date = datetime(target_year, target_month, 1).strftime("%Y-%m-%d")
    url = f"https://m.sports.naver.com/basketball/schedule/index?category={category}&date={date}"
    driver.get(url)
    time.sleep(2)
    elements = driver.find_elements(By.CSS_SELECTOR, ".ScheduleLeagueType_title__2Kalm")
    match_elements = driver.find_elements(By.CSS_SELECTOR, ".ScheduleLeagueType_match_list__1-n6x")
    for date_element, match_element in zip(elements, match_elements):
        print(f"\n[{date_element.text}] 경기일정:")  # 날짜 출력
        # li 요소들 찾기
        li_elements = match_element.find_elements(By.CSS_SELECTOR, "li")
        for idx, li in enumerate(li_elements):
            match_time = li.find_element(By.CSS_SELECTOR, ".MatchBox_time__nIEfd")
            match_status = li.find_element(By.CSS_SELECTOR, ".MatchBox_status__2pbzi")
            match_area = li.find_element(By.CSS_SELECTOR, ".MatchBox_match_area__39dEr")
            match_areas = match_area.find_elements(By.CSS_SELECTOR, ".MatchBoxTeamArea_team__3aB4O")
            link = match_area.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
            actual_time = match_time.get_attribute("textContent").strip().split("\n")[-1][-5:]

            print(f"경기날짜: {date_element.text}")
            print(f"경기 시간: {actual_time}")
            print(f"경기 상태: {match_status.text}")
            print(f"매치 정보: 홈 {match_areas[0].text} vs 원정 {match_areas[1].text}")
            print(f"응원 링크: {link}/cheer")
            print("-" * 50)
    time.sleep(1)


[11월 1일 (금)] 경기일정:
경기날짜: 11월 1일 (금)
경기 시간: 09:00
경기 상태: 종료
매치 정보: 홈 밀워키 vs 원정 멤피스
응원 링크: https://m.sports.naver.com/game/2024103129/cheer
--------------------------------------------------
경기날짜: 11월 1일 (금)
경기 시간: 09:30
경기 상태: 종료
매치 정보: 홈 휴스턴 vs 원정 댈러스
응원 링크: https://m.sports.naver.com/game/2024103106/cheer
--------------------------------------------------
경기날짜: 11월 1일 (금)
경기 시간: 10:00
경기 상태: 종료
매치 정보: 홈 샌안토니오 vs 원정 유타
응원 링크: https://m.sports.naver.com/game/2024103126/cheer
--------------------------------------------------
경기날짜: 11월 1일 (금)
경기 시간: 11:30
경기 상태: 종료
매치 정보: 홈 피닉스 vs 원정 LA클리퍼스
응원 링크: https://m.sports.naver.com/game/2024103112/cheer
--------------------------------------------------

[11월 2일 (토)] 경기일정:
경기날짜: 11월 2일 (토)
경기 시간: 08:00
경기 상태: 종료
매치 정보: 홈 올랜도 vs 원정 클리블랜드
응원 링크: https://m.sports.naver.com/game/2024110105/cheer
--------------------------------------------------
경기날짜: 11월 2일 (토)
경기 시간: 08:00
경기 상태: 종료
매치 정보: 홈 뉴욕 vs 원정 디트로이트
응원 링크: https://m.sports.naver.com/game/2

KeyboardInterrupt: 

In [148]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
options = Options()
options.add_argument("--window-size=360,640") 
driver = webdriver.Chrome(options=options)
url = "https://m.sports.naver.com/game/202411230450124/cheer"
driver.get(url)
time.sleep(5)

# click_count = 0
# while True:
#     try:
#         # 명시적 대기 추가
#         more_button = WebDriverWait(driver, 10).until(
#             EC.element_to_be_clickable((By.CSS_SELECTOR, ".u_cbox_btn_more"))
#         )
        
#         # 버튼이 보이도록 스크롤
#         driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
#         time.sleep(1)
        
#         # JavaScript로 클릭 실행
#         driver.execute_script("arguments[0].click();", more_button)
        
#         click_count += 1
#         print(f"더보기 버튼 {click_count}번째 클릭 성공")
#         time.sleep(3)
        
#     except Exception as e:
#         print(f"더보기 버튼 클릭 실패: {str(e)}")
#         break



# 기존 댓글 파싱
comments = driver.find_elements(By.CSS_SELECTOR, ".u_cbox_comment")
for comment in comments:
    try:
        # 댓글 내용과 작성자 정보 추출
        content = comment.find_element(By.CSS_SELECTOR, ".u_cbox_contents").text
        author = comment.find_element(By.CSS_SELECTOR, ".u_cbox_name").text
        time2 = comment.find_element(By.CSS_SELECTOR, ".u_cbox_date").get_attribute("data-value")
        
        print(f"작성자: {author}")
        print(f"내용: {content}") 
        print(f"시간: {time2}")
        print("-" * 50)
    except Exception as e:
        print(f"댓글 파싱 중 오류 발생: {str(e)}")
        continue

# 실시간 업데이트 감지를 위한 MutationObserver 설정
script = """
const targetNode = document.querySelector('.u_cbox_list');
const config = { childList: true, subtree: true };

const callback = function(mutationsList, observer) {
    for(let mutation of mutationsList) {
        if (mutation.type === 'childList') {
            console.log('새로운 댓글이 추가되었습니다!');
            // 여기서 새로운 댓글 요소 처리
        }
    }
};

const observer = new MutationObserver(callback);
observer.observe(targetNode, config);
"""

driver.execute_script(script)

작성자: happyworld
내용: 부상조심하세요
시간: 2024-11-23T18:33:00+0900
--------------------------------------------------
작성자: 이양희
내용: ⭐️(갸)입 3 0 0 0 0 + 롤링 100⭐️(환)전 7배⭐️잎퓰세번+깊티⭐️테ㄹ : CK(7)3(3)7 카ㅌ : ⭐️CK(7)3(3)7 ⭐️
시간: 2024-11-23T18:32:59+0900
--------------------------------------------------
작성자: akdyy
내용: 조졌다 언더냐
시간: 2024-11-23T18:32:58+0900
--------------------------------------------------
작성자: 지구력
내용: 무조건 팅되도 신지현만 던져야 그게 답
시간: 2024-11-23T18:32:57+0900
--------------------------------------------------
작성자: 마이크트라웃
내용: 귀염둥이 아닌가요?
시간: 2024-11-23T18:32:57+0900
--------------------------------------------------
작성자: 스미다
내용: 김단비 더블팀 좀더 침착하게해봐 ..
시간: 2024-11-23T18:32:52+0900
--------------------------------------------------
작성자: 파이리
내용: 김진영이 에이스
시간: 2024-11-23T18:32:49+0900
--------------------------------------------------
작성자: 홍사장
내용: 한엄지슛금지시켜라
시간: 2024-11-23T18:32:46+0900
--------------------------------------------------
작성자: fog
내용: 김지영은 진짜 실력이 안느냐 매년 ㅋㅋ
시간: 2024-11-23T18:32:43+09

In [149]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

options = Options()
options.add_argument("--window-size=360,640") 
driver = webdriver.Chrome(options=options)
url = "https://m.sports.naver.com/game/202411230450124/cheer"
driver.get(url)
time.sleep(5)

def parse_comment(comment_element):
    """댓글 요소를 파싱하는 함수"""
    content = comment_element.find_element(By.CSS_SELECTOR, ".u_cbox_contents").text
    author = comment_element.find_element(By.CSS_SELECTOR, ".u_cbox_name").text
    time_posted = comment_element.find_element(By.CSS_SELECTOR, ".u_cbox_date").get_attribute("data-value")
    return {
        "author": author,
        "content": content,
        "time": time_posted
    }

# MutationObserver 스크립트 수정
script = """
const targetNode = document.querySelector('.u_cbox_list');
const config = { childList: true, subtree: true };

const callback = function(mutationsList, observer) {
    for(let mutation of mutationsList) {
        if (mutation.type === 'childList') {
            mutation.addedNodes.forEach(node => {
                if (node.classList && node.classList.contains('u_cbox_comment')) {
                    // 새로운 댓글의 정보를 수집
                    const content = node.querySelector('.u_cbox_contents').textContent;
                    const author = node.querySelector('.u_cbox_name').textContent;
                    const time = node.querySelector('.u_cbox_date').getAttribute('data-value');
                    
                    // Python으로 데이터를 전달하기 위해 window 객체에 저장
                    if (!window.newComments) window.newComments = [];
                    window.newComments.push({
                        content: content,
                        author: author,
                        time: time
                    });
                    
                    // Python에서 감지할 수 있는 커스텀 이벤트 발생
                    document.dispatchEvent(new CustomEvent('newComment'));
                }
            });
        }
    }
};

const observer = new MutationObserver(callback);
observer.observe(targetNode, config);
"""

driver.execute_script(script)

# 초기 댓글 파싱
comments = driver.find_elements(By.CSS_SELECTOR, ".u_cbox_comment")
for comment in comments:
    try:
        comment_data = parse_comment(comment)
        print(f"작성자: {comment_data['author']}")
        print(f"내용: {comment_data['content']}") 
        print(f"시간: {comment_data['time']}")
        print("-" * 50)
    except Exception as e:
        print(f"댓글 파싱 중 오류 발생: {str(e)}")

# 새로운 댓글 감지 및 처리
def check_new_comments():
    while True:
        try:
            # JavaScript에서 저장한 새로운 댓글 데이터 가져오기
            new_comments = driver.execute_script("return window.newComments || [];")
            if new_comments:
                # 새로운 댓글 처리 후 배열 비우기
                driver.execute_script("window.newComments = [];")
                for comment in new_comments:
                    print("\n새로운 댓글이 추가되었습니다!")
                    print(f"작성자: {comment['author']}")
                    print(f"내용: {comment['content']}")
                    print(f"시간: {comment['time']}")
                    print("-" * 50)
            time.sleep(1)  # CPU 사용량 감소를 위한 짧은 대기
        except KeyboardInterrupt:
            print("\n모니터링을 종료합니다.")
            break
        except Exception as e:
            print(f"오류 발생: {str(e)}")
            continue

# 새로운 댓글 모니터링 시작
check_new_comments()

작성자: BOHEMIAN KO
내용: 우리은행이 클라스가 다르네
시간: 2024-11-23T18:33:35+0900
--------------------------------------------------
작성자: dssr
내용: 단비누나
시간: 2024-11-23T18:33:34+0900
--------------------------------------------------
작성자: 메밀꽃필무렵
내용: 에어단비 ㅠㅠ 미쳤다
시간: 2024-11-23T18:33:34+0900
--------------------------------------------------
작성자: 보이스카웃
내용: 단브론 ㄷㄷ
시간: 2024-11-23T18:33:33+0900
--------------------------------------------------
작성자: Welcome
내용: 둘의 차이 ㅋ
시간: 2024-11-23T18:33:30+0900
--------------------------------------------------
작성자: 홍사장
내용: 갓단시
시간: 2024-11-23T18:33:26+0900
--------------------------------------------------
작성자: LG야구광팬
내용: 단비 222
시간: 2024-11-23T18:33:25+0900
--------------------------------------------------
작성자: happyworld
내용: 찬스다 !
시간: 2024-11-23T18:33:23+0900
--------------------------------------------------
작성자: fog
내용: 신한은 글렀다 꼬라지 보니까
시간: 2024-11-23T18:33:22+0900
--------------------------------------------------
작성자: dssr
내용: 저걸 ㅡㅡ
시간: 2024-11-23T18:33:22+0900
------